# MXNet/Gluon Inference

In [1]:
import os
import sys
import numpy as np
import mxnet as mx
from mxnet import gluon, nd
from collections import namedtuple
from common.params_inf import *
from common.utils import *

In [2]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.14.1
GPU:  ['Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB']
CUDA Version 8.0.61
CuDNN Version  6.0.21


In [4]:
Batch = namedtuple('Batch', ['data'])

In [5]:
# Create batches of fake data
fake_input_data_cl, fake_input_data_cf = give_fake_data(BATCH_SIZE*BATCHES_GPU)
print(fake_input_data_cl.shape, fake_input_data_cf.shape)

(1280, 224, 224, 3) (1280, 3, 224, 224)


In [6]:
# Download Resnet weights
path='http://data.mxnet.io/models/imagenet/'
mx.test_utils.download(path+'resnet/50-layers/resnet-50-symbol.json')
mx.test_utils.download(path+'resnet/50-layers/resnet-50-0000.params')
print("Downloaded")

Downloaded


In [7]:
# Load model
sym, arg_params, aux_params = mx.model.load_checkpoint('resnet-50', 0)
# List the last 10 layers
all_layers = sym.get_internals()

In [8]:
# Get last layer
flatten_layer = all_layers['flatten0_output']

In [9]:
# We create the network
net = gluon.nn.SymbolBlock(outputs=flatten_layer, inputs=mx.sym.var('data'))
ctx = mx.gpu()

In [10]:
# We assign the weights
net_params = net.collect_params()
for param in arg_params:
    if param in net_params:
        net_params[param]._load_init(arg_params[param], ctx=ctx)
for param in aux_params:
    if param in net_params:
        net_params[param]._load_init(aux_params[param], ctx=ctx)

In [11]:
# We hybridize the network
net.hybridize()

In [12]:
def predict_fn(classifier, data, batchsize):
    """ Return features from classifier """
    out = nd.zeros((len(data), RESNET_FEATURES), dtype=np.float32)
    for idx, dta in yield_mb_X(data, batchsize):
        outputs = classifier(mx.nd.array(dta, ctx=ctx))
        out[idx*batchsize:(idx+1)*batchsize] = outputs
    nd.waitall()
    return out.asnumpy()

In [13]:
cold_start = predict_fn(net, fake_input_data_cf, BATCH_SIZE)

In [14]:
%%time
features = predict_fn(net, fake_input_data_cf, BATCH_SIZE)

CPU times: user 2.1 s, sys: 630 ms, total: 2.73 s
Wall time: 2.2 s


In [16]:
print("Images per second {}".format((BATCH_SIZE*BATCHES_GPU)/2.2))

Images per second 581.8181818181818
